## Etapa 2: Engenharia de Features (Avançada)

Neste notebook, vamos carregar os dados consolidados e criar as features (sinais) que o nosso modelo de IA irá usar para aprender a prever a compatibilidade entre um candidato e uma vaga.

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import json

### Bloco 1: Carregar os dados consolidados do JSON

In [8]:
df = pd.read_json('../data/processed/dados_consolidados.json', lines=True)

### Bloco 2: Preparação dos Textos

In [9]:
cols_vaga = [
    'informacoes_basicas.titulo_vaga', 
    'perfil_vaga.principais_atividades', 
    'perfil_vaga.competencia_tecnicas_e_comportamentais'
]
cols_candidato = [
    'cv_pt', 
    'informacoes_profissionais.conhecimentos_tecnicos', 
    'informacoes_profissionais.area_atuacao'
]

for col in cols_vaga + cols_candidato:
    if col in df.columns:
        df[col] = df[col].fillna('')

df['texto_vaga'] = df[cols_vaga].agg(' '.join, axis=1).str.lower()
df['texto_candidato'] = df[cols_candidato].agg(' '.join, axis=1).str.lower()

### Bloco 3: Criação das Features

In [10]:
# Feature 1: Similaridade de Texto (NLP)
vectorizer = TfidfVectorizer()
corpus = df['texto_vaga'].tolist() + df['texto_candidato'].tolist()
tfidf_matrix = vectorizer.fit_transform(corpus)
vaga_vectors = tfidf_matrix[0:len(df)]
candidato_vectors = tfidf_matrix[len(df):]
df['similaridade_texto'] = [cosine_similarity(vaga_vectors[i], candidato_vectors[i])[0][0] for i in range(len(df))]

# Features de Compatibilidade de Idioma e SAP
mapa_niveis = {'Nenhum': 0, 'Básico': 1, 'Intermediário': 2, 'Técnico': 3, 'Avançado': 4, 'Fluente': 5}
df['vaga_nivel_ingles_num'] = df['perfil_vaga.nivel_ingles'].fillna('Nenhum').apply(lambda x: mapa_niveis.get(x, 0))
df['candidato_nivel_ingles_num'] = df['formacao_e_idiomas.nivel_ingles'].fillna('Nenhum').apply(lambda x: mapa_niveis.get(x, 0))
df['match_nivel_ingles'] = (df['candidato_nivel_ingles_num'] >= df['vaga_nivel_ingles_num']).astype(int)
df['vaga_nivel_espanhol_num'] = df['perfil_vaga.nivel_espanhol'].fillna('Nenhum').apply(lambda x: mapa_niveis.get(x, 0))
df['candidato_nivel_espanhol_num'] = df['formacao_e_idiomas.nivel_espanhol'].fillna('Nenhum').apply(lambda x: mapa_niveis.get(x, 0))
df['match_nivel_espanhol'] = (df['candidato_nivel_espanhol_num'] >= df['vaga_nivel_espanhol_num']).astype(int)
df['vaga_requer_sap'] = (df['informacoes_basicas.vaga_sap'] == 'Sim').astype(int)
df['candidato_tem_sap'] = df['texto_candidato'].str.contains('sap', flags=re.IGNORECASE, regex=True).astype(int)
df['match_sap'] = ((df['vaga_requer_sap'] == 0) | ((df['vaga_requer_sap'] == 1) & (df['candidato_tem_sap'] == 1))).astype(int)

# Feature: Match de Anos de Experiência
def extract_years_experience(text):
    if not isinstance(text, str): return 0
    matches = re.findall(r'(\d{1,2})\s+an[o|os]', text, re.IGNORECASE)
    years = [int(year) for year in matches]
    return max(years) if years else 0
df['vaga_anos_exp'] = df['texto_vaga'].apply(extract_years_experience)
df['candidato_anos_exp'] = df['texto_candidato'].apply(extract_years_experience)
df['match_anos_experiencia'] = (df['candidato_anos_exp'] >= df['vaga_anos_exp']).astype(int)

# Feature: Score de Habilidades Técnicas
SKILLS_LIST = [
        'python', 'java', 'javascript', 'typescript', "c#", 'c++', 'php', 'ruby', 'go', 'swift', 'kotlin', 'sql', 'pl/sql',
        'react', 'angular', 'vue', 'svelte', 'jquery', 'node.js', 'django', 'flask', 'spring', 'ruby on rails', '.net', 'laravel',
        'aws', 'azure', 'google cloud', 'gcp', 'oracle cloud', 'oci', 'mysql', 'postgresql', 'mongodb', 'redis', 'oracle', 
        'sql server', 'dynamodb', 'docker', 'kubernetes', 'jenkins', 'git', 'github', 'gitlab', 'ansible', 'terraform', 'ci/cd',
        'agile', 'scrum', 'kanban', 'api', 'rest', 'graphql', 'microservices', 'linux', 'unix', 'html', 'css', 'sap'
    ]
def calculate_skills_match(vaga_text, candidate_text):
    required_skills = {skill for skill in SKILLS_LIST if skill in vaga_text}
    if not required_skills:
        return 1.0 
    matched_skills = sum(1 for skill in required_skills if skill in candidate_text)
    return matched_skills / len(required_skills)
df['skills_match_score'] = df.apply(lambda row: calculate_skills_match(row['texto_vaga'], row['texto_candidato']), axis=1)

# Feature: Match de Nível Profissional
mapa_senioridade = {'júnior': 1, 'junior': 1, 'pleno': 2, 'sênior': 3, 'senior': 3, 'especialista': 4, 'analista': 2}
def get_seniority_level(text):
    if not isinstance(text, str): return 0
    text = text.lower()
    for level, value in mapa_senioridade.items():
        if level in text:
            return value
    return 0
df['vaga_nivel_prof'] = df['perfil_vaga.nivel profissional'].fillna('').apply(get_seniority_level)
df['candidato_nivel_prof'] = df['informacoes_profissionais.nivel_profissional'].fillna('').apply(get_seniority_level)
df['match_nivel_profissional'] = (df['candidato_nivel_prof'] >= df['vaga_nivel_prof']).astype(int)

### Bloco 3A: Novas Features Avançadas

In [11]:
# Feature: Match de Títulos de Cargo
def get_title_keywords(title):
    if not isinstance(title, str): return set()
    # Remove palavras comuns e foca nos substantivos e adjetivos
    stopwords = {'de', 'do', 'da', 'para', 'em', 'com'}
    keywords = {word.lower() for word in re.split(r'\s+|/|-', title) if word.lower() not in stopwords and len(word) > 2}
    return keywords

def calculate_title_match(vaga_title, candidate_experiences):
    vaga_keywords = get_title_keywords(vaga_title)
    if not vaga_keywords: return 0.0
    
    candidate_titles_text = ""
    if isinstance(candidate_experiences, list):
        for exp in candidate_experiences:
            if isinstance(exp, dict) and 'cargo_atual' in exp:
                candidate_titles_text += ' ' + str(exp['cargo_atual'])
    
    candidate_keywords = get_title_keywords(candidate_titles_text)
    if not candidate_keywords: return 0.0
    
    intersection = len(vaga_keywords.intersection(candidate_keywords))
    return intersection / len(vaga_keywords)

df['match_titulo_cargo'] = df.apply(lambda row: calculate_title_match(row.get('informacoes_basicas.titulo_vaga'), row.get('informacoes_profissionais.experiencias')), axis=1)

# Feature: Score de Soft Skills
SOFT_SKILLS_LIST = ['comunicação', 'liderança', 'proatividade', 'negociação', 'organização', 'flexibilidade', 'criatividade', 'resolução de problemas', 'trabalho em equipe']
def calculate_soft_skills_match(vaga_text, candidate_text):
    required_skills = {skill for skill in SOFT_SKILLS_LIST if skill in vaga_text}
    if not required_skills: return 1.0
    
    matched_skills = sum(1 for skill in required_skills if skill in candidate_text)
    return matched_skills / len(required_skills)

df['soft_skills_match_score'] = df.apply(lambda row: calculate_soft_skills_match(row['texto_vaga'], row['texto_candidato']), axis=1)

### Bloco Final: Seleção e Salvamento

In [12]:
features_selecionadas = [
    'id_vaga',
    'prospect.codigo',
    'match',
    'similaridade_texto',
    'match_nivel_ingles',
    'match_nivel_espanhol',
    'match_sap',
    'match_anos_experiencia',
    'skills_match_score',
    'match_nivel_profissional',
    'match_titulo_cargo', # <-- Nova Feature
    'soft_skills_match_score' # <-- Nova Feature
]
df_features = df[features_selecionadas]

df_features.to_json('../data/processed/dados_com_features.json', orient='records', lines=True, force_ascii=False)
print("Arquivo 'dados_com_features.json' salvo com sucesso com as novas features!")

Arquivo 'dados_com_features.json' salvo com sucesso com as novas features!
